# Vary parameters for a calcsfh call

This example will show how to use scripts.vary_matchparam on example data to run a simple parameter sweep.

The code was written to interate what <code>calcsfh</code> doesn't. I.e., dmod, Av are varied according to the calcsfh parameter file. This code takes over if you wanted to vary IMF, dAv, BF, or model subdirectory by writing many calcsfh calls to a bash script.

## Prepare the calcsfh parameter file
See the MATCH README file for more information. The dmod, Av, CMD limits, and age bins should all be constant for the template.

It's better to leave off the IMF value in the template paramter file, that is, the first line should have only 6 numbers (<code>dmod0, dmod1, ddmod, Av0, Av1, dAv</code>). 
<code>vary_matchparam</code>, will insert the powerlaw slope if needed, but it [currenty] wont delete if if -kroupa or -chabrier are called as flags.

Here's our example template:

In [11]:
! cat 9891_NGC1978_F555W_F814W.gst.param

 18.3 18.6 0.05 0.05 0.25 0.05
-1.55 -0.45 0.07
0.0 0.000001 0.000001
1
0.10 0.05 5 -0.2 1.7 WFC555W,WFC814W
17.6 27.0 WFC555W
16.5 24.1 WFC814W
0 0
8
     9.25 9.30
     9.30 9.35
     9.35 9.40
     9.40 9.45
     9.45 9.50
     9.50 9.55
     9.55 9.60
     9.60 9.65


The second line is <code>logZ0, logZ1</code> and <code>dlogZ</code>. If you are running <code>calcsfh</code> without the <code>-ssp</code> flag, see the README file for other options for this line. 

Notes:
* <code>logZ = log10(Z/Zsun)</code>; and <code>Zsun</code> is defined in the<code>Models/makemod.cpp</code> file.
* <code>dlogZ</code> is the dispersion in Z of the model SSPs
* The logZ0, logZ1 and the ages all need to be within the boundaries of the models. If the terminal output of the calcsfh run has an error <code>"Models need to be made for ... "</code> check that the corresponding file exists as <code>Models/data/mod*_Age0_Age1_logZ+4_dlogZ</code>.


## vary_matchparam

Typing <code>python -m match.scripts.vary_matchparam -h<code> into the command line gives the options:

In [39]:
! python -m match.scripts.vary_matchparam -h

/Users/rosenfield/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
usage: vary_matchparam.py [-h] [-n NPROC] [-o OUTFILE] [-i IMF] [-b BF]
                          [-a DAV] [-s SUB] [-e EXTRA] [-d DESTINATION] [-c]
                          param_file phot fake

vary BF, IMF, and dAv

positional arguments:
  param_file            template parameter file
  phot                  match photometry file
  fake                  match ast file

optional arguments:
  -h, --help            show this help message and exit
  -n NPROC, --nproc NPROC
                        number of simultaneous calls to calcsfh
  -o OUTFILE, --outfile OUTFILE
                        file to save the script
  -i IMF, --imf IMF     IMF min, max, dIMF
  -b BF, --bf BF        BF min, max, dBF
  -a DAV, --dav DAV     dAv min, max, ddAv
  -s SUB, --sub SU

### Example Usage:
The following will write 12 calcsfh calls at a time to <code>calcsfh_ssp.sh</code> and will vary the binary fraction from 0-0.55 at 0.1 incriments, the differential extinction from 0-0.55 at 0.1 incriments, and keep the IMF as kroupa.

In [40]:
! python -m match.scripts.vary_matchparam -n 12 -i kroupa -b 0,.55,0.1 -a 0,0.55,0.1 9891_NGC1978_F555W_F814W.gst.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake

/Users/rosenfield/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
appended calcsfh_ssp.sh


Since <code>IMF</code> and <code>BF</code> are written in the parameter file, a new one parameter file is written for each combination. Note the original parameter file was not changed.

In [17]:
ls *param

9891_NGC1978_F555W_F814W.gst.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.0.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.1.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.2.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.3.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.4.param
9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.5.param


However, <code>dAv</code>, <code>sub</code> are not written to the calcsfh output files. So they are added to the output file names.

In [41]:
! head calcsfh_ssp.sh

/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.0.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.0_dav0.0_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.00 -kroupa > 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.0_dav0.0_ssp.scrn &
/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.1.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.1_dav0.0_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.00 -kroupa > 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.1_dav0.0_ssp.scrn &
/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.2.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.2_dav0.0_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.00 -kroupa > 9891_NGC1978_F555W_F814W.gst_imfkroupa_bf0.2_d

Note: 
* calcsfh flags and common file extensions are set in <code>config.py</code>
* The <code>wait</code> will pause the script until the above commands are completed.

## Vary the IMF as well as run with Kroupa.

Since we already made the <code>calcsfh_ssp.sh</code> file, we can simply append to it. <code>vary_matchparam</code> will append to calcsfh_ssp.sh if it exists in the directory.

In [42]:
! python -m match.scripts.vary_matchparam -n 12 -b 0,.55,0.1 -a 0,0.55,0.1 -i 0.5,1.5,0.25 9891_NGC1978_F555W_F814W.gst.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake

/Users/rosenfield/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
appended calcsfh_ssp.sh


In [36]:
ls *param

9891_NGC1978_F555W_F814W.gst.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.0.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.1.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.2.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.3.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.4.param
9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.5.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.0.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.1.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.2.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.3.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.4.param
9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.5.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.0.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.1.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.2.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.3.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.4.param
9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.5.param
9891_NGC1978_F555W_F814W.gst_imf1.25_bf0.0.param
9891_NGC1978_F555W_F814W.gst_imf1.25_bf0.1.par

Notice that even though the template parameter file had no IMF power law, vary_matchparam will insert it:

In [44]:
! head -1 9891_NGC1978_F555W_F814W.gst_imf*param

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.0.param <==
0.5 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.1.param <==
0.50 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.2.param <==
0.50 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.3.param <==
0.50 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.4.param <==
0.50 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.5_bf0.5.param <==
0.50 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.0.param <==
0.75 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.1.param <==
0.75 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.2.param <==
0.75 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.3.param <==
0.75 18.3 18.6 0.05 0.05 0.25 0.05

==> 9891_NGC1978_F555W_F814W.gst_imf0.75_bf0.4.param <==
0.75 18.3 18.6 0.05 

Notice those the IMF power law calls were appended to calcsfh_ssh.sh

In [35]:
! tail calcsfh_ssp.sh

/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.3.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.3_dav0.5_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.50 > 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.3_dav0.5_ssp.scrn &
/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.4.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.4_dav0.5_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.50 > 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.4_dav0.5_ssp.scrn &
/Volumes/raqiya/match2.6/bin/calcsfh 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.5.param 9891_NGC1978_F555W_F814W.match 9891_NGC1978_F555W_F814W.matchfake 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.5_dav0.5_ssp.out -PARSEC -dT=0.001 -dZ=0.01 -ssp -dAvy=0.0 -dAv=0.50 > 9891_NGC1978_F555W_F814W.gst_imf1.0_bf0.5_dav0.5_ssp.scrn &
/Volumes/raqiya/match2.6/bin/calcs

## Running the code
Finally, run calcsfh_ssp.sh:

    bash calcsfh_ssp.sh